# Install dependencies

In [ ]:
!pip install fiftyone==0.14.2 
!pip install ipywidgets>=7.5
!pip install tensorflow torch torchvision umap-learn
!pip install albumentations==1.1.0
!pip install opencv-python==4.5.5.62 
!pip install opencv-python-headless==4.5.4.60

# Load data

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz


In [ ]:
print(fo.list_datasets())


Using `persistent` property is useful when working in a local runtime session. Using Colab hosted runtime you will need to download the data again.

In [ ]:
classes_list = ["Cat", "Dog"]

In [ ]:
train_dataset = foz.load_zoo_dataset(
    "open-images-v6", 
    split="train", 
    label_types=["detections"], 
    classes=classes_list,
    max_samples=2000, 
    seed=51,
    shuffle=True,
    dataset_name="cat_dog_train"
)

In [ ]:
test_dataset = foz.load_zoo_dataset(
    "open-images-v6", 
    split="test", 
    label_types=["detections"], 
    classes=classes_list,
    max_samples=500, 
    seed=51,
    shuffle=True,
    dataset_name="cat_dog_test",
)

In [ ]:
val_dataset = foz.load_zoo_dataset(
    "open-images-v6", 
    split="validation", 
    label_types=["detections"], 
    classes=classes_list,
    seed=51,
    max_samples=500, 
    shuffle=True,
    dataset_name="cat_dog_val",
)

In [ ]:
from fiftyone import ViewField as F


train_dataset = fo.load_dataset("cat_dog_train").filter_labels( "detections", F("label").is_in(classes_list))
test_dataset = fo.load_dataset("cat_dog_test").filter_labels( "detections", F("label").is_in(classes_list))
val_dataset = fo.load_dataset("cat_dog_val").filter_labels( "detections", F("label").is_in(classes_list))

# Export dataset to YOLO format

In [ ]:
export_dir = "/content/Datasets/CatDogYOLO"
label_field = "detections"  

train_dataset.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split="train",
        classes=classes_list,
    )
test_dataset.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split="test",
        classes=classes_list,
    )
val_dataset.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split="validation",
        classes=classes_list,
    )

In [ ]:
import os
yaml_path = os.path.join(export_dir, 'dataset.yaml') 

In [ ]:
# Before changes
!cat {yaml_path}

### Edit YAML file

Here we have to change the yaml file:
*   add the "path" field, same as export_dir
*   change "validation" field to "val"

In Colab you can edit a file in the `files` tab on the left

**Before changes**
```
names:
- Cat
- Dog
nc: 2
test: ./images/test/
train: ./images/train/
validation: ./images/validation/
```


**After changes, file should look like this**
```
names:
- Cat
- Dog
nc: 2
path: /content/Datasets/CatDogYOLO 
test: ./images/test/
train: ./images/train/
val: ./images/validation/
```




In [ ]:
# Let's make sure the file has been successfully changed
!cat {yaml_path}

# Clone YOLOv5 repo

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install requirements

import torch
from yolov5 import utils

# Train

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train --port=6008

In [ ]:
!python3 train.py --img 416 --batch 16 --epochs 50 --data {yaml_path} --weights yolov5n.pt --cache

# Run inference on test image

In [ ]:
from IPython.display import Image

In [ ]:
# Get the first image from test directory
test_dir = os.path.join(export_dir, "images/test")
filename = os.listdir(test_dir)[0]
test_image = os.path.join(test_dir, filename)
test_image

In [ ]:
!python3 detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.25 --source {test_image}

In [ ]:
Image(filename=f'runs/detect/exp/{filename}', width=600)